In [1]:
import boto3
import sagemaker

boto3.setup_default_session(profile_name="sandbox", region_name="us-east-1")
sess = sagemaker.Session(boto_session=boto3.DEFAULT_SESSION)


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/nagarajans/Library/Application Support/sagemaker/config.yaml


PREPROCESSING JOB

In [ ]:
%pip install sagemaker

  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pathos-0.3.4-py3-none-any.whl.metadata (11 kB)
  Using cached protobuf-6.31.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached sagemaker_core-1.0.57-py3-none-any.whl.metadata (4.9 kB)
  Using cached schema-0.7.7-py2.py3-n

In [26]:
from sagemaker.processing import Processor
from sagemaker.processing import ProcessingInput, ProcessingOutput
import time

ROLE_FOR_PROCESSING_JOB = "arn:aws:iam::853973692277:role/ns2312-lr-pipeline-execution-role"
PROCESSING_JOB_CUSTOM_IMAGE_URI = "853973692277.dkr.ecr.us-east-1.amazonaws.com/ns2312-lr-preprocessing:latest"

processor = Processor(
    image_uri=PROCESSING_JOB_CUSTOM_IMAGE_URI,
    instance_count=1,
    role = ROLE_FOR_PROCESSING_JOB,
    instance_type="ml.m5.large"
)

processor.run(
    job_name=f"ns2312-housing-preprocess-job-{int(time.time())}",
    inputs=[
        ProcessingInput(source="s3://ns2312-lr-model-bucket/Housing.csv", destination="/opt/ml/processing/input")
    ],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train", destination="s3://ns2312-lr-model-bucket/HousingDataset/Training"),
        ProcessingOutput(output_name="val_data", source="/opt/ml/processing/val", destination="s3://ns2312-lr-model-bucket/HousingDataset/Validation"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test", destination="s3://ns2312-lr-model-bucket/HousingDataset/Test"),
        ProcessingOutput(output_name="scaler", source="/opt/ml/processing/scaler", destination="s3://ns2312-lr-model-bucket/scaler")
    ],
    arguments=[
        "--input-dir", "/opt/ml/processing/input",
        "--csv-name", "Housing.csv",
        "--train-dir", "/opt/ml/processing/train",
        "--val-dir", "/opt/ml/processing/val",
        "--test-dir", "/opt/ml/processing/test"
    ]
)

INFO:sagemaker:Creating processing-job with name ns2312-housing-preprocess-job-1756565712


....../opt/ml/code/preprocess.py:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[binary_cols] = df[binary_cols].replace({'yes': 1, 'no': 0}).astype(int)



TRAINING JOB

In [ ]:
from sagemaker.estimator import Estimator
import time

ROLE_FOR_TRAINING_JOB = "arn:aws:iam::853973692277:role/ns2312-lr-pipeline-execution-role"
TRAINING_JOB_CUSTOM_IMAGE_URI = "853973692277.dkr.ecr.us-east-1.amazonaws.com/ns2312-lr-training:latest"
TRAINING_DS_S3_URI = "s3://ns2312-lr-model-bucket/HousingDataset/Training/"
VALIDATION_DS_S3_URI = "s3://ns2312-lr-model-bucket/HousingDataset/Validation/"
MODEL_ARTIFACT_OUTPUT_S3_URI = "s3://ns2312-lr-model-bucket/model_output/"


estimator = Estimator(
    image_uri=TRAINING_JOB_CUSTOM_IMAGE_URI,
    role=ROLE_FOR_TRAINING_JOB,
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=MODEL_ARTIFACT_OUTPUT_S3_URI,
    base_job_name=f"ns2312-housing-training-job-{int(time.time())}"
)

estimator.set_hyperparameters(
    learning_rate=0.05,
    max_depth=6,
    n_estimators=150
)

estimator.fit({
    "train": TRAINING_DS_S3_URI,
    "val": VALIDATION_DS_S3_URI
})

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: ns2312-housing-training-job-1756541486-2025-08-30-08-11-26-410


2025-08-30 08:11:28 Starting - Starting the training job...
2025-08-30 08:12:05 Downloading - Downloading input data...
2025-08-30 08:12:30 Downloading - Downloading the training image..2025-08-30 08:12:50,141 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt
[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
2025-08-30 08:12:51,024 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-08-30 08:12:51,027 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-08-30 08:12:51,063 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-08-30 08:12:51,065 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-08-30 08:12:51,082 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-08-30 08:12:51,084 sagemaker-training-toolkit 

HYPERPARAMETER TUNING JOB

In [4]:
from sagemaker.estimator import Estimator
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, ContinuousParameter
import time

ROLE_FOR_TRAINING_JOB = "arn:aws:iam::853973692277:role/ns2312-lr-pipeline-execution-role"
TRAINING_JOB_CUSTOM_IMAGE_URI = "853973692277.dkr.ecr.us-east-1.amazonaws.com/ns2312-lr-training:latest"
TRAINING_DS_S3_URI = "s3://ns2312-lr-model-bucket/HousingDataset/Training/"
VALIDATION_DS_S3_URI = "s3://ns2312-lr-model-bucket/HousingDataset/Validation/"
MODEL_ARTIFACT_OUTPUT_S3_URI = "s3://ns2312-lr-model-bucket/model_output/"

estimator = Estimator(
    image_uri=TRAINING_JOB_CUSTOM_IMAGE_URI,
    role=ROLE_FOR_TRAINING_JOB,
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=MODEL_ARTIFACT_OUTPUT_S3_URI,
    base_job_name=f"ns2312-housing-training-job-{int(time.time())}"
)
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.01, 0.3),  
    "max_depth": IntegerParameter(3, 10),             
    "n_estimators": IntegerParameter(50, 300)        
}

objective_metric_name = "validation:rmse"


tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=[
        {"Name": "validation:rmse", "Regex": "validation:rmse=([0-9\\.]+)"}   # Must follow the same pattern as in training script 
    ],
    max_jobs=6,         
    max_parallel_jobs=2,
    objective_type="Minimize"
)


tuner.fit({
    "train": TRAINING_DS_S3_URI,
    "val": VALIDATION_DS_S3_URI
})

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


...............................................!


MODEL DEPLOYMENT

In [5]:
from sagemaker.model import Model

best_training_job = tuner.best_training_job()
print("Best training job:", best_training_job)

best_estimator = Estimator.attach(best_training_job)

model_artifact = best_estimator.model_data
print("Model artifact S3 path:", model_artifact)

ROLE_FOR_MODEL_DEPLOYMENT = "arn:aws:iam::853973692277:role/ns2312-lr-pipeline-execution-role"

model = Model(
    image_uri="853973692277.dkr.ecr.us-east-1.amazonaws.com/ns2312-lr-inference:latest", 
    model_data=model_artifact,            
    role=ROLE_FOR_MODEL_DEPLOYMENT,                            
    name="ns2312-housing-lr-model",
    sagemaker_session=sess,
    env={
        "SCALER_S3": "s3://ns2312-lr-model-bucket/scaler/scaler.pkl",
    }           
)

model.create()

Best training job: ns2312-lr-training-250830-2141-006-045efb77

2025-08-30 16:16:02 Starting - Found matching resource for reuse
2025-08-30 16:16:02 Downloading - Downloading the training image
2025-08-30 16:16:02 Training - Training image download completed. Training in progress.
2025-08-30 16:16:02 Uploading - Uploading generated training model
2025-08-30 16:16:02 Completed - Resource retained for reuse
Model artifact S3 path: s3://ns2312-lr-model-bucket/model_output/ns2312-lr-training-250830-2141-006-045efb77/output/model.tar.gz


ENDPOINT

In [6]:
predictor = model.deploy(
    initial_instance_count=1,   
    instance_type="ml.m5.large",  
    endpoint_name="ns2312-housing-lr-endpoint",
    sagemaker_session=sess,
)

Using already existing model: ns2312-housing-lr-model


---------

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 predictor = model.deploy(                                                                    │
│   2 │   initial_instance_count=1,                                                                │
│   3 │   instance_type="ml.m5.large",                                                             │
│   4 │   endpoint_name="ns2312-housing-lr-endpoint",                                              │
│                                                                                                  │
│ /Users/nagarajans/Desktop/SageMaker-Pipeline/.conda/lib/python3.11/site-packages/sagemaker/model │
│ .py:1814 in deploy                                                                               │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /Users/nagarajans/Desktop/SageMaker-Pipeline/.conda/lib/python3.11/site-packages/sagemaker/sessi │
│ on.py:6250 in endpoint_from_production_variants                                                  │
│                                                                                                  │
│   6247 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6248 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6249 │   │                                                                                     │
│ ❱ 6250 │   │   return self.create_endpoint(                                                      │
│   6251 │   │   │   endpoint_name=name,                                                           │
│   6252 │   │   │   config_name=name,                                                             │
│   6253 │   │   │   tags=endpoint_tags,                                                           │
│                                                                                                  │
│ /Users/nagarajans/Desktop/SageMaker-Pipeline/.conda/lib/python3.11/site-packages/sagemaker/sessi │
│ on.py:5084 in create_endpoint                                                                    │
│                                                                                                  │
│   5081 │   │   │   │   self.endpoint_arn = res["EndpointArn"]                                    │
│   5082 │   │   │                                                                                 │
│   5083 │   │   │   if wait:                                                                      │
│ ❱ 5084 │   │   │   │   self.wait_for_endpoint(endpoint_name, live_logging=live_logging)          │
│   5085 │   │   │   return endpoint_name                                                          │
│   5086 │   │   except Exception as e:                                                            │
│   5087 │   │   │   troubleshooting = (                     

In [8]:
%pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sagemaker.s3 import S3Downloader
import joblib
import os
s3_uri = "s3://ns2312-lr-model-bucket/scaler/scaler.pkl"
print(s3_uri)
local_dir = "./tmp"
S3Downloader.download(s3_uri, local_dir)
scaler_local_path = os.path.join(local_dir, "scaler.pkl")
scaler = joblib.load(scaler_local_path)

s3://ns2312-lr-model-bucket/scaler/scaler.pkl


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 local_dir = "./tmp"                                                                         │
│    7 S3Downloader.download(s3_uri, local_dir)                                                    │
│    8 scaler_local_path = os.path.join(local_dir, "scaler.joblib")                                │
│ ❱  9 scaler = joblib.load(scaler_local_path)                                                     │
│   10                                                                                             │
│                                                                                                  │
│ /Users/nagarajans/Desktop/SageMaker-Pipeline/.conda/lib/python3.11/site-packages/joblib/numpy_pi │
│ ckle.py:735 in load                                                                              │
│                                                                                                  │
│   732 │   │   with _validate_fileobject_and_memmap(fobj, filename, mmap_mode) as (fobj, _):      │
│   733 │   │   │   obj = _unpickle(fobj, ensure_native_byte_order=ensure_native_byte_order)       │
│   734 │   else:                                                                                  │
│ ❱ 735 │   │   with open(filename, "rb") as f:                                                    │
│   736 │   │   │   with _validate_fileobject_and_memmap(f, filename, mmap_mode) as (              │
│   737 │   │   │   │   fobj,                                                                      │
│   738 │   │   │   │   validated_mmap_mode,                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: './tmp/scaler.joblib'